In [1]:
from autograd import grad
import autograd.numpy as np
import matplotlib.pyplot as plt
from lista import LISTA, lasso_loss, logreg_loss
from scipy.linalg import hadamard

In [2]:
n_samples = 1000
n_test = 100
n = 10
p = 8
model = 'logreg'

loss = {'lasso': lasso_loss, 'logreg': logreg_loss}[model]

H = hadamard(p) / np.sqrt(p)
#S = p / np.arange(1, p + 1)
#S **= 2
S = np.ones(p)
K = np.dot(H, S[:, None] * H.T)

z_true = np.random.randn(p, n_samples)
D = np.random.randn(n, p).dot(K)
D /= np.linalg.norm(D, axis=0, keepdims=True)

sigma = 0.1
eps = np.random.randn(n, n_samples)

z_test = np.random.randn(p, n_test)
eps_test = np.random.randn(n, n_test)
X = np.dot(D, z_true) + eps
X_test = np.dot(D, z_test) + eps_test
if model == 'logreg':
    X = 2 * (X > 0) - 1
    X_test = 2 * (X_test > 0) - 1
lbda_max = np.max(np.abs(D.T.dot(X)))
lbda = lbda_max * 0.1
print(X.shape)

(10, 1000)


In [3]:
ista = LISTA(D, lbda, 1000, model=model)
print(loss(np.zeros((p, n_samples)), X, D, lbda))
f_star = loss(ista.transform(X), X, D, lbda)
print(f_star)

6.931471805599452
5.349208015822353


In [4]:
n_layers = 10
lista = LISTA(D, lbda, n_layers, model=model, l_star=f_star)
loss(lista.transform(X), X, D, lbda)

5.359089638596476

In [5]:
lista.fit(X, l_rate=1e-1, max_iter=1000, batch_size=1000, verbose=True)

it 0, loss = 9.88e-03, grad = 2.79e-02
it 100, loss = 7.28e-03, grad = 8.88e-03
it 200, loss = 6.96e-03, grad = 6.63e-03
it 300, loss = 6.88e-03, grad = 6.37e-03
it 400, loss = 6.85e-03, grad = 6.42e-03
it 500, loss = 6.84e-03, grad = 6.37e-03
it 600, loss = 6.83e-03, grad = 6.39e-03
it 700, loss = 6.83e-03, grad = 6.37e-03
it 800, loss = 6.83e-03, grad = 6.39e-03
it 900, loss = 6.83e-03, grad = 6.39e-03


In [6]:
lista_one = LISTA(D, lbda, 1, model=model)
lista_one.fit(X, l_rate=1e-2, max_iter=10000, batch_size=1000, verbose=True)

it 0, loss = 5.78e+00, grad = 1.71e-01
it 100, loss = 5.75e+00, grad = 1.60e-01
it 200, loss = 5.73e+00, grad = 1.49e-01
it 300, loss = 5.70e+00, grad = 1.40e-01
it 400, loss = 5.69e+00, grad = 1.31e-01
it 500, loss = 5.67e+00, grad = 1.24e-01
it 600, loss = 5.66e+00, grad = 1.17e-01
it 700, loss = 5.64e+00, grad = 1.10e-01
it 800, loss = 5.63e+00, grad = 1.04e-01
it 900, loss = 5.62e+00, grad = 9.90e-02
it 1000, loss = 5.61e+00, grad = 9.41e-02
it 1100, loss = 5.60e+00, grad = 8.96e-02
it 1200, loss = 5.60e+00, grad = 8.53e-02
it 1300, loss = 5.59e+00, grad = 8.15e-02
it 1400, loss = 5.58e+00, grad = 7.78e-02
it 1500, loss = 5.58e+00, grad = 7.44e-02
it 1600, loss = 5.57e+00, grad = 7.14e-02
it 1700, loss = 5.57e+00, grad = 6.83e-02
it 1800, loss = 5.56e+00, grad = 6.56e-02
it 1900, loss = 5.56e+00, grad = 6.31e-02
it 2000, loss = 5.55e+00, grad = 6.07e-02
it 2100, loss = 5.55e+00, grad = 5.80e-02
it 2200, loss = 5.55e+00, grad = 5.54e-02
it 2300, loss = 5.54e+00, grad = 5.33e-02
it 2

In [7]:
f_star = loss(LISTA(D, lbda, 1000, model=model).transform(X_test), X_test, D, lbda)
f0 = loss(np.zeros_like(z_test), X_test, D, lbda)

In [10]:
ista = LISTA(D, lbda, n_layers, model=model)
print('Avg test loss using ISTA with %d iterations: %.2e' %
      (n_layers, (loss(ista.transform(X_test), X_test, D, lbda) - f_star) / (f0 - f_star)))

Avg test loss using ISTA with 10 iterations: 6.88e-03


In [11]:
print('Avg test loss using LISTA with %d layers: %.2e' %
      (n_layers, (loss(lista.transform(X_test), X_test, D, lbda) - f_star) / (f0 - f_star)))

Avg test loss using LISTA with 10 layers: 5.11e-03
